In [1]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import importlib

import spacy

from utilities import _support
from utilities._support import ig_profiles

from nltk.tokenize import word_tokenize, sent_tokenize

importlib.reload(_support)

<module 'utilities._support' from 'c:\\Users\\tommaso\\Documents\\_Projects\\IGScraper\\utilities\\_support.py'>

In [11]:
with open('utilities/lookup_users.pkl', 'rb') as f:
    full_lookup = pickle.load(f)

df = pd.read_json('gitshared_csv/matteosalviniofficial/matteosalviniofficial_comments_20230316_0-1100_preprocessed.json')